In [ ]:
#pip install pandas
import pandas as pd
import datetime as dt
import numpy as np
#data
df_airports = pd.read_csv(path+'airports.csv')
df_airlines = pd.read_csv(path+'airlines.csv')
df_hdata = pd.read_csv(path+'historic_data.csv', low_memory=False)
df_hdata = df_hdata .sample(frac=0.10)
#drop cancelled flights
df_delays=df_hdata[df_hdata['CANCELLED']==0]

fmt="%Y/%m/%d %H:%M:%S"
fmt2="%H:%M:%S"
##############
#to build dates on each flight stage
#############

# 1) add date to departure time
df_delays['DEP_date_time']=pd.to_datetime(df_hdata['SCHEDULED_DEPARTURE'], format=fmt)+pd.to_timedelta(df_delays['DEPARTURE_DELAY'],'m')

# 2) arrival_time: DEP_date_time+ tax_out (min)+airtime (min)+tax_in (min)
df_delays['Arr_date_time']=df_delays['DEP_date_time']+ pd.to_timedelta(df_delays['TAXI_IN'],'m')+pd.to_timedelta(df_delays['TAXI_OUT'],'m')+ pd.to_timedelta(df_delays['AIR_TIME'],'m')

#3)compute the date of schedule arrival: SCHEDULED_DEPARTURE+SCHEDULED_TIME (min)
df_delays['SCH_ARR_date_time']=pd.to_datetime(df_hdata['SCHEDULED_DEPARTURE'], format=fmt)+pd.to_timedelta(df_delays['SCHEDULED_TIME'],'m')
#4) compute arrival delay: schedule arrival-Arr_date_time
df_delays['arrival_delay']=(df_delays['SCH_ARR_date_time']-df_delays['Arr_date_time']).astype('timedelta64[m]')
# to compare with the initial delay version
df_delays['arrival_delayv2']=(pd.to_datetime(df_delays['SCHEDULED_ARRIVAL'], format=fmt2)- pd.to_datetime(df_delays['ARRIVAL_TIME'], format=fmt2)).astype('timedelta64[m]')